In [7]:
import cv2, dlib, sys
import numpy as np

scaler = 0.4
cam = cv2.VideoCapture(0)
cam.set(3,1280)
cam.set(4,720)
detector = dlib.get_frontal_face_detector()#얼굴 디렉터 모듈 초기화
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#load video
#cap = cv2. VideoCapture(0)

while True:
    ret_val, img = cam.read()
    #cv2.imread(file, cv2.IMREAD_NUMCHANGED) -> file이미지를 BGRA타입으로 읽기

    ret,img = cam.read()
    if not ret_val:
        break;
        
    img = cv2.resize(img, (int(img.shape[1]*scaler), int(img.shape[0]*scaler)))#동영상 사이즈 너무 커서 resize
    ori = img.copy()
    
    #detect faces
    faces = detector(img)
    face = faces[0]
    
    dlib_shape = predictor(img,face) #img의 face영역 안의 얼굴 특징점 찾기
    shape_2d = np.array([[p.x,p.y] for p in dlib_shape.parts()]) #연산 쉽게하기 위함
    
    #compute center and boundaries of face 
    #특징점찾은 이유 : 얼굴의 좌상단, 우하단, 얼굴의 사이즈, 중심 구하기
    top_left = np.min(shape_2d, axis=0)
    bottom_right = np.max(shape_2d, axis=0)
   
    #우하단에서 좌상단 좌표를 뺀 (x,y 길이의 가장 긴 값)
    face_size = int(max(bottom_right - top_left)*2.5)
    
    
    #모든 특징점의 평균 구해서 얼굴 중앙 찾기 : np.mean()
    center_x, center_y = np.mean(shape_2d,axis=0).astype(np.int)
    #소수점일수도 있어서 int형으로 변경 : np.astype(np.int) 
    
    #visualize
    img = cv2.rectangle(img, pt1=(face.left(), face.top()), pt2=(face.right(), face.bottom()), color=(255,255,255),thickness=2, lineType=cv2.LINE_AA)
    
    for s in shape_2d: #얼굴 특징점 68개
        cv2.circle(img, center=tuple(s), radius=1, color=(255,255,255), thickness=2, lineType=cv2.LINE_AA)
    
    
    #전체 좌표값
    overall_vertices = []
    for x in range(0,68):
        coordination = tuple(shape_2d[x])
        overall_vertices.append(coordination + (1,)) # z좌표 값 추가
    overall_vertices = tuple(overall_vertices)
    
    
    #얼굴형 좌표값
    face_vertices = []
    for x in range(0,17):
        coordination = tuple(shape_2d[x])
        face_vertices.append(coordination) # z좌표 값 추가
    face_vertices = tuple(face_vertices)

     #왼쪽눈썹 좌표값
    leftEB_vertices = []
    for x in range(17,22):
        coordination = tuple(shape_2d[x])
        leftEB_vertices.append(coordination) # z좌표 값 추가
    leftEB_vertices = tuple(leftEB_vertices)
    
    #오른쪽눈썹 좌표값
    rightEB_vertices = []
    for x in range(22,27):
        coordination = tuple(shape_2d[x])
        rightEB_vertices.append(coordination) # z좌표 값 추가
    rightEB_vertices = tuple(rightEB_vertices)
    
    #왼쪽눈 좌표값
    leftEye_vertices = []
    for x in range(36,42):
        coordination = tuple(shape_2d[x])
        leftEye_vertices.append(coordination) # z좌표 값 추가
    leftEye_vertices = tuple(leftEye_vertices)
    
    #오른쪽눈 좌표값
    rightEye_vertices = []
    for x in range(42,48):
        coordination = tuple(shape_2d[x])
        rightEye_vertices.append(coordination) # z좌표 값 추가
    rightEye_vertices = tuple(rightEye_vertices)
    
    #코 좌표값
    nose_vertices = []
    for x in range(27,36):
        coordination = tuple(shape_2d[x])
        nose_vertices.append(coordination) # z좌표 값 추가
    nose_vertices = tuple(nose_vertices)
    
    #입 좌표값
    mouse_vertices = []
    for x in range(48,68):
        coordination = tuple(shape_2d[x])
        mouse_vertices.append(coordination) # z좌표 값 추가
    mouse_vertices = tuple(mouse_vertices)
    
    #입 바깥
    mouseOut_vertices = []
    for x in range(48,60):
        coordination = tuple(shape_2d[x])
        mouseOut_vertices.append(coordination) # z좌표 값 추가
    mouseOut_vertices = tuple(mouseOut_vertices)
    
    #특징점으로 만든 얼굴의 사이즈 -> 원그리기
    #cv2.circle(img, center=tuple(top_left), radius=1, color=(255,0,0), thickness=2, lineType=cv2.LINE_AA)
    #cv2.circle(img, center=tuple(bottom_right), radius=1, color=(255,0,0), thickness=2, lineType=cv2.LINE_AA)
    #cv2.circle(img, center=tuple((center_x, center_y)), radius=1, color=(0,0, 255), thickness=2, lineType=cv2.LINE_AA)
    
    #얼굴형 렌더링
    for i in range(0, 16):
        cv2.line(img, face_vertices[i], face_vertices[i+1], (0,255,0)) 
    
    #왼쪽 눈썹
    for i in range(0, 4):
        cv2.line(img, leftEB_vertices[i], leftEB_vertices[i+1], (0,255,0), 10)
    #오른쪽 눈썹
    for i in range(0, 4):
        cv2.line(img, rightEB_vertices[i], rightEB_vertices[i+1], (0,255,0), 10) 
    
    #왼쪽눈
    for i in range(0, 5):
        cv2.line(img, leftEye_vertices[i], leftEye_vertices[i+1], (0,255,0))
    cv2.line(img, leftEye_vertices[0], leftEye_vertices[5], (0,255,0))
    
    #오른쪽 눈
    for i in range(0, 5):
        cv2.line(img, rightEye_vertices[i], rightEye_vertices[i+1], (0,255,0))
    cv2.line(img, rightEye_vertices[0], rightEye_vertices[5], (0,255,0))
    
    #코
    for i in range(0, 8):
        cv2.line(img, nose_vertices[i], nose_vertices[i+1], (0,255,0))
    cv2.line(img, nose_vertices[3], nose_vertices[8], (0,255,0))
    #입
    for i in range(0, 19):
        cv2.line(img, mouse_vertices[i], mouse_vertices[i+1], (0,0,255), 5)
    cv2.line(img, mouse_vertices[19], mouse_vertices[0], (0,0,255), 5)
        
    cv2.imshow('video',img)
    if cv2.waitKey(1) == 27:
        break

KeyboardInterrupt: 